In [ ]:
# Roboflow API for dataset retrieval

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uVOAXSWN9rfXjBxgvL5W")
project = rf.workspace("titaniumsv5").project("ssa-annotation")
version = project.version(6)
dataset = version.download("coco-segmentation")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SSA-Annotation-6 in coco-segmentation:: 100%|██████████| 85/85 [00:00<00:00, 709.85it/s]


In [2]:
pip install pycocotools opencv-python

In [3]:
!ls /content/SSA-Annotation-6/valid/

_annotations.coco.json
img_adjusted_Raw_Observation_004_Set1_png.rf.b27b3221d82ae9165b4b635351134e93.jpg
img_adjusted_Raw_Observation_005_Set1_png.rf.a0c6c601d292cc1341b8fd75700b009f.jpg
img_adjusted_Raw_Observation_010_Set1_png.rf.bf72c59d596abbd60f39527a5701c2bd.jpg
img_adjusted_Raw_Observation_011_Set1_png.rf.f65db8188bac8789889801ceb947d0e8.jpg
img_adjusted_Raw_Observation_013_Set2_png.rf.466ea8b1fa305b477cdda5b7128a53e4.jpg
img_adjusted_Raw_Observation_021_Set2_png.rf.6067dd87aa6967dee7a369722dcc0a6e.jpg
img_adjusted_Raw_Observation_026_Set3_png.rf.9fe188f9a48c00ea23fcc03b47a79fd6.jpg
img_adjusted_Raw_Observation_029_Set3_png.rf.099e49918eed5baa232a5fed736abbe8.jpg
img_adjusted_Raw_Observation_031_Set3_png.rf.526bd807285b63a719da265bca3618f4.jpg
img_adjusted_Raw_Observation_032_Set3_png.rf.01e0ef3fa4abcdfc153f6ee5cacdcd59.jpg


In [ ]:
from torchvision import transforms

# Image transformation
transform = transforms.Compose([
    transforms.Resize((128, 128)),   
    transforms.ToTensor(),    
])

In [ ]:
import os
import json
import numpy as np
from PIL import Image
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from pycocotools.coco import COCO

# Dataset (COCO segmentation format)
class CocoSegmentationDataset(Dataset):
    def __init__(self, image_dir, annotation_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.coco = COCO(annotation_file)
        self.image_ids = list(self.coco.imgs.keys())

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
      img_id = self.image_ids[idx]
      img_info = self.coco.loadImgs(img_id)[0]
      img_path = os.path.join(self.image_dir, img_info['file_name'])

      image = cv2.imread(img_path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = Image.fromarray(image)

    # Creating mask
      ann_ids = self.coco.getAnnIds(imgIds=img_id)
      anns = self.coco.loadAnns(ann_ids)
      mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
      for ann in anns:
        mask = np.maximum(mask, self.coco.annToMask(ann))

      if self.transform:
        image = self.transform(image)

    # Transforming mask manually
      mask = cv2.resize(mask, (image.shape[2], image.shape[1]))
      mask = torch.from_numpy(mask).float().unsqueeze(0) / 255.0

      return image, mask

In [ ]:
# Loading training dataset and creating a DataLoader for batching and shuffling

from torch.utils.data import DataLoader

train_dataset = CocoSegmentationDataset(
    image_dir="/content/SSA-Annotation-6/train",
    annotation_file="/content/SSA-Annotation-6/train/_annotations.coco.json",
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Basic UNet architecture for image segmentation

class UNet(nn.Module):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        
        # Encoder blocks for downsampling path (Input/Output)
        self.enc1 = self.contract_block(3, 64)
        self.enc2 = self.contract_block(64, 128)
        self.enc3 = self.contract_block(128, 256)

        # Bottleneck block for lowest paart of network
        self.bottleneck = self.double_conv(256, 512)

        # Decoder blocks for upsampling path
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.double_conv(256 + 128, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.double_conv(128 + 64, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.double_conv(64, 64)

        # Final convolution layer
        self.final_conv = nn.Conv2d(64, num_classes, kernel_size=1)

    # Contracting block for downsampling path
    def contract_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    # Double conv block, no downsampling
    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)

        bottleneck = self.bottleneck(enc3)

        up3 = self.up3(bottleneck)
        dec3 = self.dec3(torch.cat([up3, enc2], dim=1))

        up2 = self.up2(dec3)
        dec2 = self.dec2(torch.cat([up2, enc1], dim=1))

        up1 = self.up1(dec2)
        dec1 = self.dec1(up1)

        return self.final_conv(dec1)


# Model training loop

num_classes = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(num_classes=num_classes).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss() 

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, masks in train_loader:
        images = images.to(device)                  
        masks = masks.to(device).long().squeeze(1)  
        optimizer.zero_grad()
        outputs = model(images)                      
        loss = criterion(outputs, masks)   
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/20], Loss: 1.0354
Epoch [2/20], Loss: 1.0098
Epoch [3/20], Loss: 0.8309
Epoch [4/20], Loss: 0.0276
Epoch [5/20], Loss: 0.0000
Epoch [6/20], Loss: 0.0000
Epoch [7/20], Loss: 0.0000
Epoch [8/20], Loss: 0.0000
Epoch [9/20], Loss: 0.0000
Epoch [10/20], Loss: 0.0000
Epoch [11/20], Loss: 0.0000
Epoch [12/20], Loss: 0.0000
Epoch [13/20], Loss: 0.0000
Epoch [14/20], Loss: 0.0000
Epoch [15/20], Loss: 0.0000
Epoch [16/20], Loss: 0.0000
Epoch [17/20], Loss: 0.0000
Epoch [18/20], Loss: 0.0000
Epoch [19/20], Loss: 0.0000
Epoch [20/20], Loss: 0.0000


In [ ]:
# Define transform for validation dataset (same as training)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

val_dataset = CocoSegmentationDataset(
    image_dir="/content/SSA-Annotation-6/valid", 
    annotation_file="/content/SSA-Annotation-6/valid/_annotations.coco.json",
    transform=transform
)

validation_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
